# Quantitative evaluation

## Setup

### Modules

In [4]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy.stats as spst

import tractseg.config as config
import tractseg.data.dataset_specific_utils

### Data

In [ ]:
DATASET = "HCP105"
CLASSSET = "bundles_all72"
CVFOLD = 0

EXPERIMENTS = ["peaks", "rank_3_approx", "fodfs"]
FILENAME_EXPERIMENT = "experiment_test.pkl"

path_dir_experiments = Path(config.PATH_DIR_EXP)
paths_experiments = [path_dir_experiments / experiment for experiment in EXPERIMENTS]

### Utilities

In [ ]:
def loadall(path):
    with open(path, "rb") as f:
        while True:
            try:
                yield pickle.load(f)
            except EOFError:
                break


def load_experiment(path):
    experiment = loadall(path)
    return experiment

## Plotting results

In [ ]:
subjects = tractseg.data.dataset_specific_utils.get_subjects(DATASET)
subjects_train, subjects_validate, subjects_test = tractseg.data.dataset_specific_utils.get_cvfold(DATASET, CVFOLD)
classes = tractseg.data.dataset_specific_utils.get_classes(CLASSSET)

In [ ]:
scores_peaks = np.array([score[0] for score in metrics_bundles_peaks.values()])
scores_fodfs = np.array([score[0] for score in metrics_bundles_fodfs.values()])
scores_rank3 = np.array([score[0] for score in metrics_bundles_rank3.values()])


ids_order = np.argsort(scores_peaks)

names = np.array(list(metrics_bundles_peaks.keys()))
names = names[ids_order]
BUNDLES_ALL72 = np.array(BUNDLES_ALL72)
names = names_order[np.array([np.where(BUNDLES_ALL72 == query)[0] for query in names])]
scores_peaks = scores_peaks[ids_order]
scores_fodfs = scores_fodfs[ids_order]
scores_rank3 = scores_rank3[ids_order]

In [ ]:
dina4 = np.array([0.21, 0.297]) * 39.37
fig = plt.figure(figsize=dina4)

plt.scatter(scores_peaks, names.ravel(), marker="o", label="Peaks")
plt.scatter(scores_rank3, names.ravel(), marker="+", label="Rank-3 approximation")
plt.scatter(scores_fodfs, names.ravel(), marker="*", label="fODFs")
plt.xlabel("Dice")
plt.ylabel("Bundle")
plt.grid(alpha=0.6)
plt.legend()
plt.tight_layout()
plt.show()

## Statistical tests

In [ ]:
spst.friedmanchisquare()